# TP2 - DB Normalization and Querying (By Daniel Jorge Deutsch)

The objectives of this TP are:
1. Apply normalization 1NF -> 2NF -> 3NF
2. Perform SQL queries on the normalized database

In this TP, we will use a database **`wine.db`** (available in the course's website) containing wine information related to 'production' and 'sales'. 

> Production <---> Wine <---> Sales


---

### Working with db files in Jupyter
- Python provides an interface for SQLite through the *sqlite3* module
- The **`%%sql`** magic builds upon it (and other tools) to enable the usage of SQL commands within a Jupyter Notebook as in common SQL clients.
- Before proceeding, make sure that **`wine.db`** is in the same path as this notebook.
  - If **`wine.db`** is not in the same path, an empty **`wine.db`** file will be created, resulting in errors in later steps of the TP.
- The SQLite module in Python commits transactions automatically, this means that any change in the DB is immediately written to the file, e.g. creating/deleting tables.
  -  For this reason, it is recommended to have a backup of **`wine.db`** as it is provided in the course's website.

---

**`wine.db`** contains the following unnormalized tables:

<center>**Master1**</center>

|*Attribute*|         *Description*          |
| -------   |--------------------------------|
| NV        | Wine number                    |
| CRU       | Vineyard or group of vineyards |
| DEGRE     | Alcohol content                |
| MILL      | Vintage year                   |
| QTE       | Number of bottles harvested    |
| NP        | Producer number                |
| NOM       | Producer's last name           |
| PRENOM    | Producer's first name          |
| REGION    | Production region              |

From wikipedia:

__Cru__: Often used to indicate a specifically named and legally defined vineyard or ensemble of vineyards and the vines "which grow on [such] a reputed terroir; by extension of good quality." The term is also used to refer to the wine produced from such vines.


<center>**Master2**</center>

|*Attribute*|                         *Description*                  |
| -------   |--------------------------------------------------------|
| NV        | Wine number                                            |
| CRU       | Vineyard or group of vineyards                         |
| DEGRE     | Alcohol content                                        |
| MILL      | Vintage year                                           |
| DATES     | Buying date                                            |
| LIEU      | Place where the wine was sold                          |
| QTE       | Number of bottles bought                               |
| NB        | Client (buveur) number                                 |
| NOM       | Client's last name                                     |
| PRENOM    | Client's first name                                    |
| TYPE      | Type of client by volume of purchases                  |
| REGION    | Administrative Region (different to production region) |


In [1]:
import sqlite3    # Python interface for SQLite databases

In [2]:
def printSchema(connection):
    # Function to print the DB schema
    # Source: http://stackoverflow.com/a/35092773/4765776
    for (tableName,) in connection.execute(
        """
        select NAME from SQLITE_MASTER where TYPE='table' order by NAME;
        """
    ):
        print("{}:".format(tableName))
        for (
            columnID, columnName, columnType,
            columnNotNull, columnDefault, columnPK,
        ) in connection.execute("pragma table_info('{}');".format(tableName)):
            print("  {id}: {name}({type}){null}{default}{pk}".format(
                id=columnID,
                name=columnName,
                type=columnType,
                null=" not null" if columnNotNull else "",
                default=" [{}]".format(columnDefault) if columnDefault else "",
                pk=" *{}".format(columnPK) if columnPK else "",
            ))

In [3]:
conn = sqlite3.connect('wine.db')
c = conn.cursor()
print("Database schema:")
printSchema(conn)           # An usefull way to viualize the content of the database

Database schema:
LOCATION:
  0: LIEU(TEXT)
  1: REGION(TEXT)
MASTER1:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)
  4: QTE(NUM)
  5: NP(NUM)
  6: NOM(TEXT)
  7: PRENOM(TEXT)
  8: REGION(TEXT)
MASTER2:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)
  4: DATES(DATE)
  5: LIEU(TEXT)
  6: QTE(NUM)
  7: NB(NUM)
  8: NOM(TEXT)
  9: PRENOM(TEXT)
  10: TYPE(TEXT)
  11: REGION(TEXT)
client:
  0: NB(NUM)
  1: NOM(TEXT)
  2: PRENOM(TEXT)
  3: TYPE(TEXT)
producer:
  0: NP(NUM)
  1: NOM(TEXT)
  2: PRENOM(TEXT)
  3: REGION(TEXT)
production:
  0: NP(NUM)
  1: NV(NUM)
  2: QTE(NUM)
purchase:
  0: NB(NUM)
  1: NV(NUM)
  2: DATES(NUM)
  3: LIEU(TEXT)
  4: QTE(NUM)
wine:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)


From this point we will use __%%sql__ magic

In [4]:
%load_ext sql
%sql sqlite:///wine.db

# PART I: Database normalization

The first task on this TP is the normalization of the wine data. In its current state both tables **Master1** and **Master2** are in the First Normal Form (1NF).

By inspecting the content of these tables we can see that multiple tuples have NULL values.

In [5]:
%%sql SELECT *
FROM Master1
LIMIT 10;

 * sqlite:///wine.db
Done.


NV,CRU,DEGRE,MILL,QTE,NP,NOM,PRENOM,REGION
None,None,None,None,None,3,Six,Paul,Alsace
None,None,None,None,None,6,Marmagne,Bernard,Bourgogne
None,None,None,None,None,8,Lioger d'Harduy,Gabriel,Bourgogne
None,None,None,None,None,16,Barbin,Bernard,Bourgogne
None,None,None,None,None,17,Faiveley,Guy,Bourgogne
None,None,None,None,None,18,Tramier,Jean,Bourgogne
None,None,None,None,None,19,Dupaquier,Roger,Bourgogne
None,None,None,None,None,20,Lamy,Jean,Bourgogne
None,None,None,None,None,21,Cornu,Edmond,Bourgogne
None,None,None,None,None,26,Violot,Gilbert,Bourgogne


* Notice that Jupyter *displays* 'None' instead of 'NULL'. 
  - This is only to comply with python notation.
* To account for NULL values, your SQL queries must test explicitly for 'NULL'.

Another problem in **Master1** and **Master2** is data redundancy, for example:

In [6]:
%%sql SELECT *
FROM Master1
WHERE NV = 45;

 * sqlite:///wine.db
Done.


NV,CRU,DEGRE,MILL,QTE,NP,NOM,PRENOM,REGION
45,Chiroubles,None,1983,90,2,Boxler,Albert,Alsace
45,Chiroubles,None,1983,912,67,Descombes,Jean Ernest,Beaujolais
45,Chiroubles,None,1983,98,71,Chalandard,Danile,Jura
45,Chiroubles,None,1983,540,78,Michlel,Pierre Emile,Jura
45,Chiroubles,None,1983,450,86,Dumazet,Marc,Rhone


---

Additional resource for Normalization:

https://www.youtube.com/watch?v=UrYLYV7WSHM

---

#### Exercise 1.1

Convert table **Master1** to the Second Normal Form (2NF) or Third Normal Form (3NF) as applicable.
* Explain your answer
* List main functional dependencies (not all of them)
* Describe the schema of new tables and how they relate
  * You can write Tables as above or you can insert images in the notebook.
  
Remember that **`wine.db`** contains information related to wine 'production' and 'sells'.

> Production <---> Wine <---> Sales

A good start point is to look for the 'Wine' attributes.

**Hint:** Look for redundant information between the master tables.

**1NF:**

We notice that the table Master1, with key (NV, NP), is already in the First Normal Form since:

- The values in each cell are atomic (can't be splitted);
- The values in each column follow the same data type;
- Rows are uniquely identified;

<br>

**2NF:**

To adequate the information on table Master1 to the Second Normal Form, we should:

- Respect the 1NF;
- Every attribute (i.e. every non-key value) should be dependent on the key;

<br>

**3NF:**

Finally, to the Third Normal Form, we should have:

- All columns can be determined only by the key of the table;

<br>

**Functional Dependencies:**

Firstly, we notice that CRU, DEGRE and MILL are attributes that depend only on the wine itself, so they must be set on a separete table alongside with NV (the table key). Then, we realize that NOM, PRENOM and REGION are attributes related to the wine producer, therefore, they should also be moved to a different table with the key being NP. Finally, we see that QTE depends on both the wine and the producer, so it should be on a table with key (NV, NP).

- {NP} -> {NOM, PRENOM, REGION}: given the producer number of the wine, we should be able to identify the producer's attributes such as its first name, last name and the production region;
- {NV} -> {CRU, DEGRE, MILL}: given the wine number, we should be able to identify the vineyard that produced the wine grapes, the amount of alcohol in the wine and its vintage year;
- {NV, NP} -> {QTE}: given the wine and the producer numbers, we should be able to identify the number of bottles harvest; 

<br>

**Solution:**

Considering all that, to respect the 3NF, we could use the following schemas:

*Producer table:*

| *Attribute* | *Status*              | *Description*               |
|-------------|----------------------:|----------------------------:|
| NP          | Key                   | Producer number             |
| NOM         |                       | Producer's last name        |
| PRENOM      |                       | Producer's first name       |
| REGION      |                       | Production region           |

*Wine table:*

| *Attribute* | *Status*              | *Description*                        |
|-------------|----------------------:|-------------------------------------:|
| NV          | Key                   | Wine number                          |
| CRU         |                       | Vineyard or group of vineyards       |
| DEGRE       |                       | Alcohol content                      |
| MILL        |                       | Vintage year                         |

*Production table:*

| *Attribute* | *Status*              | *Description*                     |
|-------------|----------------------:|----------------------------------:|
| NV          | Key                   | Wine number                       |
| NP          | Key                   | Producer number                   |
| QTE         |                       | Number of bottles harvested       |

#### Exercise 1.2

Convert table **Master2** to the Second Normal Form (2NF) or Third Normal Form (3NF) as applicable.
* Explain your answer
* List main functional dependencies (not all of them)
* Describe the schema of new tables and how they relate
  * You can write Tables as above or you can insert images in the notebook.

**Note:** For this part, consider that a wine can be bought in multiple locations and multiple times per day.

**Functional Dependencies:**

Following the definitions of the 1NF, 2NF and 3NF defined in the previous exercise, we can see that the table Master2 (with key (NB, NV)) is already in the 1NF.

Firstly we notice that now, instead of information about the wine producer, we have information about the wine buyer. This, to respect the 3NF should be placed on a different table with key NB. Then, we notice that on the table Master2 we have the same information about wine that was present on the table Master1 (NV, CRU, DEGRE, MILL), so it should be on a separate table as stated in the previous exercise. Then, we notice that we have information about the client's purchase (the client, wine, QTE, DATE and LIEU). Finally, we can see that the REGION can be easily identified by the LIEU, so it should be on a separate table and the purchase table should use a foreignkey to identify the REGION.

- {NV} -> {CRU, DEGRE, MILL}: given the wine number, we should be able to identify the vineyard that produced the wine grapes, the amount of alcohol in the wine and its vintage year;
- {NB} -> {NOM, PRENOM, TYPE}: given the client number, we should be able to identify attributes related to the client such as its first and last name and its type;
- {NV, NB, DATE, LIEU} -> {QTE}: for one to know the amount of wine bought, it should have information about the client, the wine and the place and time the selling occured (a wine can be bought in multiple locations and multiple times per day);
- {LIEU} -> {REGION}: given the location where the wine was sold, one should be able to identify its region;
 
<br>

**Solution:**

Considering all that, to respect the 3NF, we could use the following schemas:

*Wine table:*

| *Attribute* | *Status*              | *Description*                        |
|-------------|----------------------:|-------------------------------------:|
| NV          | Key                   | Wine number                          |
| CRU         |                       | Vineyard or group of vineyards       |
| DEGRE       |                       | Alcohol content                      |
| MILL        |                       | Vintage year                         |

*Client table:*

| *Attribute* | *Status*              | *Description*                               |
|-------------|----------------------:|--------------------------------------------:|
| NB          | Key                   | Client (buveur) number                      |
| NOM         |                       | Client's last name                          |
| PRENOM      |                       | Client's first name                         |
| TYPE        |                       | Type of client by volume of purchases       |

*Purchase table:*

| *Attribute* | *Status*              | *Description*                       |
|-------------|----------------------:|------------------------------------:|
| NV          | Key                   | Wine number                         |
| NB          | Key                   | Client (buveur) number              |
| DATES       | Key                   | Buying date                         |
| LIEU        | Key                   | Place where the wine was sold       |
| QTE         |                       | Number of bottles bought            |

*Region table:*

| *Attribute* | *Status*              | *Description*                                                |
|-------------|----------------------:|-------------------------------------------------------------:|
| LIEU        | Key                   | Place where the wine was sold                                |
| REGION      | Key                   | Administrative Region (different to production region)       |


Once you have defined the 2NF or 3NF (as applicable) we need to split the data into new tables.

A table can be created from the result of a query.

In the following example we will create a new table "dummy" to store the different values of alcohol content.

In [7]:
%%sql DROP TABLE IF EXISTS dummy;

-- Create dummy table
CREATE TABLE dummy AS
SELECT DISTINCT DEGRE
FROM MASTER1;

 * sqlite:///wine.db
Done.
Done.


[]

In [8]:
print("\nContent of the database")
printSchema(conn)


Content of the database
LOCATION:
  0: LIEU(TEXT)
  1: REGION(TEXT)
MASTER1:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)
  4: QTE(NUM)
  5: NP(NUM)
  6: NOM(TEXT)
  7: PRENOM(TEXT)
  8: REGION(TEXT)
MASTER2:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)
  4: DATES(DATE)
  5: LIEU(TEXT)
  6: QTE(NUM)
  7: NB(NUM)
  8: NOM(TEXT)
  9: PRENOM(TEXT)
  10: TYPE(TEXT)
  11: REGION(TEXT)
client:
  0: NB(NUM)
  1: NOM(TEXT)
  2: PRENOM(TEXT)
  3: TYPE(TEXT)
dummy:
  0: DEGRE(NUM)
producer:
  0: NP(NUM)
  1: NOM(TEXT)
  2: PRENOM(TEXT)
  3: REGION(TEXT)
production:
  0: NP(NUM)
  1: NV(NUM)
  2: QTE(NUM)
purchase:
  0: NB(NUM)
  1: NV(NUM)
  2: DATES(NUM)
  3: LIEU(TEXT)
  4: QTE(NUM)
wine:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)


In [9]:
%%sql
SELECT *
FROM dummy;

 * sqlite:///wine.db
Done.


DEGRE
None
11.5
11.3
12.1
10.9
11.7
11.2
12.3
11.9
11.8


Looking into "dummy", we notice that our query includes NULL. This is not allowed if we were to use DEGRE as key for a table.

To correct this, we need to change the query to explicitly test for NULL as follows:

In [10]:
%%sql DROP TABLE IF EXISTS dummy;

-- Create dummy table
CREATE TABLE dummy AS
SELECT DISTINCT DEGRE
FROM MASTER1
WHERE DEGRE IS NOT NULL;

SELECT *
FROM dummy;

 * sqlite:///wine.db
Done.
Done.
Done.


DEGRE
11.5
11.3
12.1
10.9
11.7
11.2
12.3
11.9
11.8
10.7


Notice that we use `NULL` given that `None` is only used for display.

In [11]:
# Remove "dummy" table
%sql DROP TABLE IF EXISTS dummy;

 * sqlite:///wine.db
Done.


[]

#### Exercise 1.3

Create the new tables from Master1:

In [12]:
%%sql 

DROP TABLE IF EXISTS producer;

CREATE TABLE producer AS
SELECT DISTINCT 
    NP, NOM, PRENOM, REGION
FROM MASTER1
WHERE 
    NP IS NOT NULL;

 * sqlite:///wine.db
Done.
Done.


[]

In [13]:
%%sql

DROP TABLE IF EXISTS wine;

CREATE TABLE wine AS
SELECT DISTINCT 
    NV, CRU, DEGRE, MILL
FROM 
    MASTER1
WHERE 
    NV IS NOT NULL;

 * sqlite:///wine.db
Done.
Done.


[]

In [14]:
%%sql

DROP TABLE IF EXISTS production;

CREATE TABLE production AS
SELECT DISTINCT
    NP, NV, QTE
FROM 
    MASTER1
WHERE 
    NV IS NOT NULL 
    OR NP IS NOT NULL;

 * sqlite:///wine.db
Done.
Done.


[]

#### Exercise 1.4

Create the new tables from Master2:

In [15]:
%%sql 

DROP TABLE IF EXISTS client;

CREATE TABLE client AS
SELECT DISTINCT 
    NB, NOM, PRENOM, TYPE
FROM 
    MASTER2
WHERE 
    NB IS NOT NULL;

 * sqlite:///wine.db
Done.
Done.


[]

In [16]:
%%sql

DROP TABLE IF EXISTS purchase;

CREATE TABLE purchase AS
SELECT DISTINCT 
    NB, NV, DATES, LIEU, QTE 
FROM MASTER2
WHERE 
    NB IS NOT NULL
    AND NV IS NOT NULL
    AND DATES IS NOT NULL
    AND LIEU IS NOT NULL;

 * sqlite:///wine.db
Done.
Done.


[]

In [17]:
# It is not necessary to create the wine table from the Master2 table because it would have the same schema and content as the wine table created from the Master1 table.

In [18]:
%%sql 

DROP TABLE IF EXISTS LOCATION;

CREATE TABLE LOCATION AS
SELECT DISTINCT 
    LIEU, REGION
FROM MASTER2
WHERE 
    LIEU IS NOT NULL;

 * sqlite:///wine.db
Done.
Done.


[]

# PART II: SQL QUERIES

In the second part of this TP you will create SQL queries to retrieve information from the database.

**Important:**

- You MUST use the normalized tables created in previous steps.
  - The normalized tables will also be used in TP3.
- Do NOT use **Master1** and **Master2** in your queries.

#### Exercise 2.1

What are the different types of clients (buveurs) by volume of purchases?

In [19]:
%%sql

SELECT DISTINCT 
    TYPE
FROM client;

 * sqlite:///wine.db
Done.


TYPE
petit
moyen
gros


#### Exercise 2.2

What regions produce Pommard or Brouilly?

In [20]:
%%sql

SELECT DISTINCT
    CRU, REGION
FROM
    wine, producer, production
WHERE 
    (wine.CRU = 'Pommard' OR wine.CRU = 'Brouilly')
    AND producer.NP = production.NP
    AND production.NV = wine.NV;

 * sqlite:///wine.db
Done.


CRU,REGION
Pommard,Bourgogne
Pommard,Rhone
Brouilly,Bourgogne


#### Exercise 2.3

What regions produce Pommard and Brouilly?

In [21]:
%%sql

SELECT 
    REGION 
FROM 
    wine, producer, production
WHERE 
    wine.CRU = 'Pommard'
    AND producer.NP = production.NP
    AND production.NV = wine.NV
INTERSECT
SELECT 
    REGION 
FROM 
    wine, producer, production
WHERE 
    wine.CRU = 'Brouilly'
    AND producer.NP = production.NP
    AND production.NV = wine.NV;

 * sqlite:///wine.db
Done.


REGION
Bourgogne


#### Exercise 2.4

Get the number of wines bught by CRU and Millésime

In [22]:
%%sql

SELECT 
    CRU, MILL, SUM(QTE) AS N_WINES
FROM 
    wine, purchase
WHERE 
    wine.NV = purchase.NV
GROUP BY 
    CRU, MILL
ORDER BY 
    CRU;

 * sqlite:///wine.db
Done.


CRU,MILL,N_WINES
Arbois,1980,8
Auxey Duresses,1914,80
Beaujolais Primeur,1983,7
Beaujolais Villages,1975,10
Beaujolais Villages,1976,120
Beaujolais Villages,1978,130
Beaujolais Villages,1979,520
Chapelle Chambertin,1973,30
Chateau Corton Grancey,1980,4
Chenas,1984,1


#### Exercise 2.5

Retrieve the wine number (NV) of wines produced by more than three producers

In [23]:
%%sql

SELECT 
    NV, COUNT(NP) AS N_PRODUCERS
FROM 
    production
WHERE 
    NV IS NOT NULL
GROUP BY 
    NV
HAVING 
    COUNT(NP) > 3;

 * sqlite:///wine.db
Done.


NV,N_PRODUCERS
45,5
78,5
89,4
98,5


#### Exercise 2.6

Which producers have not produced any wine?

In [24]:
%%sql 

SELECT 
    producer.NP, producer.NOM, producer.PRENOM 
FROM 
    producer
LEFT JOIN production ON 
    producer.NP = production.NP
WHERE 
    production.NV IS NULL;

 * sqlite:///wine.db
Done.


NP,NOM,PRENOM
3,Six,Paul
6,Marmagne,Bernard
8,Lioger d'Harduy,Gabriel
16,Barbin,Bernard
17,Faiveley,Guy
18,Tramier,Jean
19,Dupaquier,Roger
20,Lamy,Jean
21,Cornu,Edmond
26,Violot,Gilbert


#### Exercise 2.7

What clients (buveurs) have bought at least one wine from 1980?

In [25]:
%%sql 

SELECT DISTINCT
    client.NB, client.NOM, client.PRENOM
FROM 
    client, purchase, wine
WHERE
    client.NB = purchase.NB
    AND wine.MILL = '1980'
    AND wine.NV = purchase.NV
ORDER BY 
    client.NB;

 * sqlite:///wine.db
Done.


NB,NOM,PRENOM
2,Artaud,Antonin
8,Aragon,Louis
44,Gide,Andre
45,Giono,Jean
50,Lautreamont,None
61,Mallarme,Stephane


#### Exercise 2.8

What clients (buveurs) have NOT bought any wine from 1980?

In [26]:
%%sql 

SELECT DISTINCT 
    NB, NOM, PRENOM
FROM 
    client
EXCEPT
SELECT DISTINCT
    client.NB, client.NOM, client.PRENOM
FROM 
    client, purchase, wine
WHERE
    client.NB = purchase.NB
    AND purchase.NV = wine.NV
    AND wine.MILL = '1980'
ORDER BY
    client.NB;

 * sqlite:///wine.db
Done.


NB,NOM,PRENOM
1,Aristote,None
3,Aron,Raymond
4,Apollinaire,Guillaume
5,Audiberti,Jacques
6,Arrabal,Fernando
7,Anouilh,Jean
9,Ajar,Emile
10,Andersen,Yann
11,Breton,Andre
12,Bataille,Georges


#### Exercise 2.9

What clients (buveurs) have bought ONLY wines from 1980?

In [27]:
%%sql

SELECT DISTINCT 
    client.NB, client.NOM, client.PRENOM
FROM 
    client, purchase, wine
WHERE 
    wine.MILL = '1980'
    AND wine.NV = purchase.NV
    AND purchase.NB = client.NB
EXCEPT
SELECT DISTINCT 
    client.NB, client.NOM, client.PRENOM
FROM 
    client, purchase, wine
WHERE 
    wine.MILL NOT IN ('1980')
    AND wine.NV = purchase.NV
    AND purchase.NB = client.NB
ORDER BY 
    client.NB;

 * sqlite:///wine.db
Done.


NB,NOM,PRENOM
44,Gide,Andre
45,Giono,Jean
50,Lautreamont,None


#### Exercise 2.10

List all wines from 1980

In [28]:
%%sql 

SELECT 
    *
FROM 
    wine
WHERE
    MILL = '1980'
ORDER BY
    NV;

 * sqlite:///wine.db
Done.


NV,CRU,DEGRE,MILL
1,Mercurey,11.5,1980
4,Mercurey,10.9,1980
16,Meursault,12.1,1980
20,Cote de Brouilly,12.1,1980
26,Chateau Corton Grancey,None,1980
28,Volnay,11,1980
43,Fleurie,11.4,1980
74,Arbois,12,1980
78,Etoile,12,1980
79,Seyssel,11,1980


#### Exercise 2.11

What are the wines from 1980 bought by NB=2?

In [29]:
%%sql 

SELECT 
    wine.NV, wine.CRU, wine.MILL, wine.DEGRE
FROM 
    wine, purchase
WHERE
    wine.MILL = '1980'
    AND purchase.NB = '2'
    AND wine.NV = purchase.NV
ORDER BY 
    wine.NV;

 * sqlite:///wine.db
Done.


NV,CRU,MILL,DEGRE
1,Mercurey,1980,11.5


#### Exercise 2.12

What clients (buveurs) have bought ALL the wines from 1980?

In [30]:
%%sql 

SELECT 
    client.NB, client.NOM, client.PRENOM
FROM 
    client, purchase, wine 
WHERE
    purchase.NB = client.NB
    AND purchase.NV = wine.NV
GROUP BY 
    client.NB
HAVING 
    COUNT(wine.NV) = ( 
        SELECT 
            COUNT(wine.NV)
        FROM 
            wine
        WHERE 
            wine.MILL = '1980'
    );

 * sqlite:///wine.db
Done.


NB,NOM,PRENOM
44,Gide,Andre
